# Assignment 7

Train a Transformer model for Machine Translation from Russian to English.  
Dataset: http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz   
Make all source and target text to lower case.  
Use following tokenization for english:  
```
import sentencepiece as spm

...
spm.SentencePieceTrainer.Train('--input=data/text.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

TGT = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),
    batch_first=True,
)

...
TGT.build_vocab(..., min_freq=5)
...

```
Score: corpus-bleu `nltk.translate.bleu_score.corpus_bleu`  
Use last 1000 sentences for model evalutation (test dataset).  
Use your target sequence tokenization for BLEU score.  
Use max_len=50 for sequence prediction.  


Hint: You may consider much smaller model, than shown in the example.  

Baselines:  
[4 point] BLEU = 0.05  
[6 point] BLEU = 0.10  
[9 point] BLEU = 0.15  

[1 point] Share weights between target embeddings and output dense layer. Notice, they have the same shape.


Readings:
1. BLUE score how to https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
1. Transformer code and comments http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls

bpe_en.model  bpe_ru.vocab  sample_data
bpe_en.vocab  drive	    training-parallel-nc-v13
bpe_ru.model  __pycache__   transformer.py


In [0]:
!pip install tqdm

In [0]:
!pip install sentencepiece

In [0]:
%load_ext autoreload
%autoreload 2
from transformer import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from tqdm import tqdm

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
#from tqdm.notebook import tqdm
from torchtext import datasets, data
#from tqdm.notebook import tqdm
import sentencepiece as spm


DEVICE = 'cuda'

In [0]:
# opening archive in google colab
# archive lies in drive/My Drive/Ru-En

!ls drive/My\ Drive/Ru-En

training-parallel-nc-v13.tgz


In [0]:
# if using google colab
import os, sys, tarfile

tar_path = 'drive/My Drive/Ru-En/training-parallel-nc-v13.tgz'
tar = tarfile.open(tar_path, 'r')

for item in tar:
    tar.extract(item, '.')

In [0]:
!ls

bpe_en.model  bpe_ru.vocab  sample_data
bpe_en.vocab  drive	    training-parallel-nc-v13
bpe_ru.model  __pycache__   transformer.py


In [0]:
# tokenize english 
with open('training-parallel-nc-v13/news-commentary-v13.ru-en.en') as f:
    with open('training-parallel-nc-v13/text.en', 'w') as out:
            out.write(f.read().lower())
        
spm.SentencePieceTrainer.Train('--input=training-parallel-nc-v13/text.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [0]:
# tokenize russian
with open('training-parallel-nc-v13/news-commentary-v13.ru-en.ru') as f:
    with open('training-parallel-nc-v13/text.ru', 'w') as out:
            out.write(f.read().lower())

spm.SentencePieceTrainer.Train('--input=training-parallel-nc-v13/text.ru --model_prefix=bpe_ru --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [0]:
tok_ru = spm.SentencePieceProcessor()
tok_ru.load('bpe_ru.model')

tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

SRC = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_ru.encode_as_pieces(x),
    batch_first=True,
)

TGT = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),
    batch_first=True,
)

fields = (('src', SRC), ('tgt', TGT))

In [0]:
with open('training-parallel-nc-v13/text.ru') as f:
    src_snt = list(map(str.strip, f.readlines()))
    
with open('training-parallel-nc-v13/text.en') as f:
    tgt_snt = list(map(str.strip, f.readlines()))
    
examples = [data.Example.fromlist(x, fields) for x in tqdm(zip(src_snt, tgt_snt))]
test = data.Dataset(examples[-1000:], fields)
train, valid = data.Dataset(examples[:-1000], fields).split(0.9)




0it [00:00, ?it/s]


405it [00:00, 4037.66it/s]


837it [00:00, 4115.36it/s]


1180it [00:00, 3880.36it/s]


1648it [00:00, 4088.72it/s]


2110it [00:00, 4232.93it/s]


2573it [00:00, 4343.01it/s]


3006it [00:00, 4337.96it/s]


3479it [00:00, 4444.61it/s]


3922it [00:00, 4438.13it/s]


4465it [00:01, 4693.07it/s]


4970it [00:01, 4793.35it/s]


5446it [00:01, 4722.12it/s]


5916it [00:01, 4669.45it/s]


6382it [00:01, 4657.83it/s]


6847it [00:01, 4591.88it/s]


7306it [00:01, 4586.93it/s]


7778it [00:01, 4624.16it/s]


8241it [00:01, 4590.19it/s]


8700it [00:01, 4524.13it/s]


9153it [00:02, 4487.94it/s]


9683it [00:02, 4701.66it/s]


10172it [00:02, 4756.10it/s]


10650it [00:02, 4710.28it/s]


11123it [00:02, 4657.31it/s]


11590it [00:02, 4588.23it/s]


12050it [00:02, 4585.70it/s]


12510it [00:02, 4576.19it/s]


12980it [00:02, 4610.88it/s]


13463it [00:02, 4674.32it/s]


13931it [00:03, 4561.80it/s]


14389it [00:03, 4438.03it/s]


14867it [00:03, 4535.05it/s]


15377it

In [0]:
print('src: ' + " ".join(train.examples[100].src))
print('tgt: ' + " ".join(train.examples[100].tgt))

src: ▁используя ▁данные ▁от ▁томаса ▁пик етти ▁и ▁ э ману и ля ▁са еса , ▁ас емо глу ▁также ▁указывает ▁на ▁то , ▁что ▁динамика ▁распределения ▁зарплаты ▁для ▁ 1% ▁тех , ▁кто ▁зарабатывает ▁больше ▁всего ▁доходов , ▁выглядит ▁иначе .
tgt: ▁using ▁data ▁from ▁thomas ▁piketty ▁and ▁emmanuel ▁sa e z , ▁acemoglu ▁also ▁points ▁out ▁that ▁the ▁dynamics ▁of ▁the ▁wage ▁distribution ▁for ▁the ▁top ▁ 1% ▁of ▁us ▁income ▁earners ▁look ▁different .


In [0]:
len(train), len(valid), len(test)

(210743, 23416, 1000)

In [0]:
TGT.build_vocab(train, min_freq=5)
SRC.build_vocab(train, min_freq=5)

In [0]:
from transformer import make_model, Batch

    
class BucketIteratorWrapper(DataLoader):
    __initialized = False

    def __init__(self, iterator: data.Iterator):
#         super(BucketIteratorWrapper,self).__init__()
        self.batch_size = iterator.batch_size
        self.num_workers = 1
        self.collate_fn = None
        self.pin_memory = False
        self.drop_last = False
        self.timeout = 0
        self.worker_init_fn = None
        self.sampler = iterator
        self.batch_sampler = iterator
        self.__initialized = True

    def __iter__(self):
        return map(
            lambda batch: Batch(batch.src, batch.tgt, pad=TGT.vocab.stoi['<pad>']),
            self.batch_sampler.__iter__()
        )

    def __len__(self):
        return len(self.batch_sampler)
    
class MyCriterion(nn.Module):
    def __init__(self, pad_idx):
        super(MyCriterion, self).__init__()
        self.pad_idx = pad_idx
        self.criterion = nn.CrossEntropyLoss(reduction='sum', ignore_index=pad_idx)
        
    def forward(self, x, target):
        x = x.contiguous().permute(0,2,1)
        ntokens = (target != self.pad_idx).data.sum()
        
        return self.criterion(x, target) / ntokens

In [0]:
!pip install transformers

In [0]:
from transformers import get_linear_schedule_with_warmup

In [0]:
torch.cuda.empty_cache()

batch_size = 64
num_epochs = 5

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train, valid, test), 
                                              batch_sizes=(batch_size, batch_size, batch_size), 
                                  sort_key=lambda x: len(x.src),
                                  shuffle=True,
                                  device=DEVICE,
                                  sort_within_batch=False)
                                  
train_iter = BucketIteratorWrapper(train_iter)
valid_iter = BucketIteratorWrapper(valid_iter)
test_iter = BucketIteratorWrapper(test_iter)


#pad_idx = TGT.vocab.stoi["<blank>"]
model = make_model(len(SRC.vocab), len(TGT.vocab), N=6, 
               d_model=256, d_ff=512, h=8, dropout=0.1)
model = model.to(DEVICE)
criterion = MyCriterion(pad_idx=TGT.vocab.stoi["<pad>"])
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)
# scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, 
#                                             num_warmup_steps=0.1*num_epochs*len(train_iter)//batch_size, 
#                                             num_training_steps=num_epochs*len(train_iter)//batch_size, 
#                                             last_epoch=-1)

# share weights

#model.src_embed[0].proj.weight = model.tgt_embed[0].lut.weight
model.generator.proj.weight = model.tgt_embed[0].lut.weight

In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [0]:
optimizer = NoamOpt(model.src_embed[0].d_model, 1, 2000,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [0]:
def train_epoch(data_iter, model, criterion):
    total_loss = 0
    #data_iter = tqdm(data_iter)
    counter = 0
    for i, batch in enumerate(data_iter):
        model.zero_grad()
        out = model.forward(batch)
        #print(torch.max(out[0], dim=-1)[1])
        loss = criterion(out, batch.tgt_y)
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss
        #data_iter.set_postfix(loss = loss)
        counter += 1
        if i % 200 == 1:
            print("Epoch Step: %d Loss: %f" %
                    (i, loss))
    return total_loss / counter


def valid_epoch(data_iter, model, criterion):
    total_loss = 0
    #data_iter = tqdm(data_iter)
    counter = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch)
        loss = criterion(out, batch.tgt_y)
        total_loss += loss
        #data_iter.set_postfix(loss = loss)
        counter += 1
        if i % 200 == 1:
            print("Epoch Step: %d Loss: %f" %
                    (i, loss))
    return total_loss / counter


for epoch in range(num_epochs):
    model.train()
    loss = train_epoch(train_iter, model, criterion)
    print('train', loss)
    
    model.eval()
    with torch.no_grad():
        loss = valid_epoch(valid_iter, model, criterion)
        scheduler.step(loss)
        print('valid', loss)

    torch.save(model.state_dict(), 'drive/My Drive/check.pt')

Epoch Step: 1 Loss: 10.249162
Epoch Step: 201 Loss: 7.140257
Epoch Step: 401 Loss: 6.318790
Epoch Step: 601 Loss: 6.029051
Epoch Step: 801 Loss: 5.550148
Epoch Step: 1001 Loss: 5.410139
Epoch Step: 1201 Loss: 5.209784


KeyboardInterrupt: ignored

In [0]:
model.load_state_dict(torch.load('drive/My Drive/check.pt'))

<All keys matched successfully>

In [0]:
start_symbol_id = TGT.vocab.stoi["<s>"]

In [0]:
def greedy_decode(model, src, src_mask, max_len):
    memory = model.encode(src, src_mask)
    ys = torch.ones(src.size(0), 1).fill_(start_symbol_id).type_as(src.data)
    for i in range(max_len-1):
        prob = model.decode((ys).long(), 
                           (subsequent_mask(ys.size(1))
                                    .type_as(src.data)),
                           memory,
                           src_mask
                           )
        _, next_word = torch.max(prob[:, -1, :], dim = -1)
        ys = torch.cat([ys, next_word.unsqueeze(-1)], dim=1)
    return ys

In [0]:
def beam_search(model, src, src_mask, max_len=10, k=5):
    memory = model.encode(src, src_mask)
    ys = torch.ones(src.size(0), 1).fill_(start_symbol_id).long().to(src.device)
    k_beam = [(0, ys)]

    for l in range(max_len):
        all_k_beams = []
        for prob, sent_pred in k_beam:
            pred = model.decode(sent_pred.long(), 
                           subsequent_mask(sent_pred.size(1))
                                    .type_as(src.data),
                           memory,
                           src_mask
                           )
            _, possible_k = torch.topk(pred[:, -1, :], k=k, dim=-1)

            all_k_beams += [
                (
                    sum([pred[0, i, sent_pred[0, i]].item() for i in range(l)]) + pred[0, -1, next_word].item(),
                    torch.cat([sent_pred, next_word.resize(sent_pred.size(0), 1)], dim=1)
                )
                for next_word in possible_k.view(k, -1)
            ]

        k_beam = sorted(all_k_beams, key=lambda x: x[0])[-k:]

    return k_beam


In [0]:
model.eval()
with torch.no_grad():
    for i, batch in enumerate(valid_iter):
        src = batch.src[:1]
        src_key_padding_mask = src != SRC.vocab.stoi["<pad>"]
        beam = beam_search(model, src, src_key_padding_mask)
        
        seq = []
        for i in range(1, src.size(1)):
            sym = SRC.vocab.itos[src[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_ru.decode_pieces(seq)
        print("\nSource:", seq)
        
        print("Translation:")
        for pred_proba, pred in beam:                
            seq = []
            for i in range(1, pred.size(1)):
                sym = TGT.vocab.itos[pred[0, i]]
                if sym == "</s>": break
                seq.append(sym)
            seq = tok_en.decode_pieces(seq)
            print(seq)
                
        seq = []
        for i in range(1, batch.tgt.size(1)):
            sym = TGT.vocab.itos[batch.tgt[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        print("Target:", seq)
        if i == 10:
            break

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



Source: рост
Translation:
growth
growth
growth
growth
growth
Target: inflation

Source: 1.
Translation:
yes.
yes.
yes.
yes.
yes.
Target: 1.

Source: они неправы.
Translation:
it is wrong.
they are wrong.
it is wrong.
they are wrong.
it is wrong.
Target: they are wrong.

Source: наверное, нет.
Translation:
maybe not.
probably not.
maybe not.
perhaps not.
maybe not.
Target: probably not.

Source: глобализация на марше
Translation:
capitalism on march
capitalism on march
capitalism on march
capitalism on march
capitalism on march
Target: globalization marches on

Source: революция в открытом образовании
Translation:
a free education revolution
a free education revolution
a free education revolution
an open education revolution
a free education revolution
Target: the open education revolution

Source: они правда хотят работать.
Translation:
it is right for the right to do so to
it is right for the right to act.
it is right for the right to work.
it is right for the right to do so.
it is r

In [0]:
from nltk.translate.bleu_score import corpus_bleu

In [0]:
hypotheses = []
references = []

model.eval()
with torch.no_grad():
    for batch in tqdm(test_iter):
        #src = batch.src
        for src, tgt in zip(batch.src, batch.tgt):
          src = src.unsqueeze(0)
          src_mask = (src != SRC.vocab.stoi["<pad>"]).unsqueeze(-2)
          out = beam_search(model, src, src_mask, 
                              max_len=40, k=5)
          
          tgt = tgt.unsqueeze(0)
          for (prob, transl), gold in zip(out, tgt):
              hyp = []
              for i in range(1, transl.size(-1)):
                  sym = TGT.vocab.itos[transl[0, i]]
                  if sym == "</s>": break
                  hyp.append(sym)
              hypotheses.append(hyp)
              
              ref = []
              for i in range(1, gold.size(-1)):
                  sym = TGT.vocab.itos[gold.data[i]]
                  if sym == "</s>": break
                  ref.append(sym)
              references.append([ref])










  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")









  6%|▋         | 1/16 [03:24<51:12, 204.84s/it]








 12%|█▎        | 2/16 [06:48<47:43, 204.53s/it]








 19%|█▉        | 3/16 [10:15<44:28, 205.26s/it]








 25%|██▌       | 4/16 [13:40<41:01, 205.16s/it]








 31%|███▏      | 5/16 [17:07<37:42, 205.67s/it]








 38%|███▊      | 6/16 [20:32<34:15, 205.60s/it]








 44%|████▍     | 7/16 [23:57<30:47, 205.32s/it]








 50%|█████     | 8/16 [27:21<27:19, 204.91s/it]








 56%|█████▋    | 9/16 [30:43<23:49, 204.14s/it]








 62%|██████▎   | 10/16 [34:06<20:21, 203.62s/it]








 69%|██████▉   | 11/16 [37:28<16:56, 203.27s/it]








 75%|███████▌  | 12/16 [40:51<13:32, 203.20s/it]








 81%|████████▏ | 13/16 [44:14<10:09, 203.07s/it]








 88%|████████▊ | 14/16 [47:36<06:45, 202.87s/it]







In [0]:
from nltk import translate

In [0]:
corpus_bleu(references, hypotheses, 
            smoothing_function=translate.bleu_score.SmoothingFunction().method3,
            auto_reweigh=True
           )

0.10282347970416357